In [103]:
# パッケージのインストール（初回のみ実行）
using Pkg
Pkg.activate(".")  # 現在のディレクトリの環境をアクティブ化
#Pkg.add("ITensors")  # ITensorsパッケージをインストール
# Pkg.add("Random")
# Pkg.add("Plots")
# Pkg.add("LaTeXStrings")

  Activating project at `c:\Users\ito23\Documents\学院科目\物性物理学特論Ⅰ\Julia\day4\Mypackage`


2次元格子状のisingモデル
\begin{equation*}
    \mathcal{H} = -J\sum_{<i,j>}\sigma_i\sigma_j - h\sum_{i}\sigma_i
\end{equation*}
書き換え
\begin{equation*}
    \mathcal{H} = -\sum_{i}\left(\frac{J}{2}\sigma_iS_i + h\sigma_i\right)
\end{equation*}
ただし$S_i$はサイト$i$の周りのスピンの和


スピンの更新ルール

スピン配位$C_k$が与えられ、あるスピンを反転した配位$C_k^\prime$をつくる

$\Delta E(C_k \to C_k^\prime)>0$のとき
$$
\exp[-\beta\Delta E]でC_{k+1} = C_k^\prime \\
1 - \exp[-\beta\Delta E]でC_{k+1} = C_k
$$
$\Delta E \leq 0$で常に$C_{k+1} = C_k^\prime$

設定 

・$50\times 50$格子、周期境界 

・総ステップ60000、熱平衡到達ステップ10000 

・測定は10001ステップから10ステップ間隔で記録、平均をとる 

・$k_BT/J=0\sim 5$まで測定

In [104]:
using Random
using Plots
using LaTeXStrings

In [105]:
# スピンの初期化

# ランダム
function spins_init(Lx, Ly, rng)
    return rand(rng, [-1, 1], Lx, Ly)
end

# 全て1
# function spins_init(Lx, Ly, rng)
#     return fill(1, Lx, Ly)
# end

spins_init (generic function with 1 method)

In [106]:
# S_iの計算
function Si_calc(Ck, ix, iy, Lx, Ly)
    jx = ix + 1
    if jx > Lx
        jx -= Lx
    end
    Si = Ck[jx, iy]

    jx = ix - 1
    if jx < 1
        jx += Lx
    end
    Si += Ck[jx, iy]

    jy = iy + 1
    if jy > Ly
        jy -= Ly
    end
    Si += Ck[ix, jy]

    jy = iy - 1
    if jy < 1
        jy += Ly
    end
    Si += Ck[ix, jy]

    return Si
end


Si_calc (generic function with 1 method)

In [107]:
# エネルギーの測定
function E_measure(Ck, J, h, Lx, Ly)
    E = 0
    for iy in 1:Ly, ix in 1:Lx
        Si = Si_calc(Ck, ix, iy, Lx, Ly)
        sigma = Ck[ix, iy]
        E += -(J / 2) * sigma * Si - h * sigma
    end
    return E
end

E_measure (generic function with 1 method)

In [108]:
# エネルギー差の計算
function delta_E_calc(Ck, J, h, ix, iy, Lx, Ly)
    Si = Si_calc(Ck, ix, iy, Lx, Ly)
    delta_E = 2 * J * Ck[ix, iy] * Si + 2 * h * Ck[ix, iy]
    return delta_E
end

delta_E_calc (generic function with 1 method)

In [109]:
# スピン相関の測定
function spin_correlation(Ck, Lx, Ly)
    mm = zeros(Lx, Ly)
    for iy in 1:Ly, ix in 1:Lx
        Si = Si_calc(Ck, ix, iy, Lx, Ly)
        mm[ix, iy] = Ck[ix, iy] * Si
    end
    return mm
end

spin_correlation (generic function with 1 method)

In [110]:
# スピンの更新
function spins_update(Ck, ix, iy, J, h, T, Lx, Ly, rng)
    delta_E = delta_E_calc(Ck, J, h, ix, iy, Lx, Ly)
    sigma = Ck[ix, iy]
    if delta_E <= 0
        return -sigma 
    else
        p = exp(-delta_E / T)
        return rand(rng) < p ? -sigma : sigma
    end
end

spins_update (generic function with 1 method)

In [111]:
# 物理量の測定
function ising_MC(step_equilibration, step_MC, measure_interval, J, h, T, Lx, Ly)
    rng = MersenneTwister(123)
    count_measure = 0
    E_mean = 0.0
    E2_mean = 0.0
    m_mean = 0.0
    abs_m_mean = 0.0
    mm_mean = 0.0

    Ck = spins_init(Lx, Ly, rng)

    for step in 1:(step_equilibration + step_MC)
        for iy in 1:Ly, ix in 1:Lx
            Ck[ix, iy] = spins_update(Ck, ix, iy, J, h, T, Lx, Ly, rng)
        end

        if step > step_equilibration && step % measure_interval == 0
            count_measure += 1
            E = E_measure(Ck, J, h, Lx, Ly) / (Lx * Ly)
            E_mean += E
            E2_mean += E^2
            m = sum(Ck) / (Lx * Ly)
            m_mean += m
            abs_m_mean += abs(m)
            mm = sum(spin_correlation(Ck, Lx, Ly)) / (4 * Lx * Ly)
            mm_mean += mm
        end
    end

    E_mean /= count_measure
    E2_mean /= count_measure
    m_mean /= count_measure
    abs_m_mean /= count_measure
    mm_mean /= count_measure
    C_mean = (E2_mean - E_mean^2) / T^2

    return E_mean, m_mean, abs_m_mean, mm_mean, C_mean
end    

ising_MC (generic function with 1 method)

In [112]:
# 実行
function run_ising_MC()
    step_equilibration = 10000
    step_MC = 50000
    measure_interval = 10
    J = 1.0
    h = 0.0
    Ts = range(0.01, 1.0, length=40)
    Lx = 50
    Ly = 50
    E_T = []
    m_T = []
    abs_m_T = []
    mm_T = []
    C_T = []

    for T in Ts
        E_mean, m_mean, abs_m_mean, mm_mean, C_mean = ising_MC(step_equilibration, step_MC, measure_interval, J, h, T, Lx, Ly)
        push!(E_T, E_mean)
        push!(m_T, m_mean)
        push!(abs_m_T, abs_m_mean)
        push!(mm_T, mm_mean)
        push!(C_T, C_mean)
    end

    plot(Ts, E_T, label="", xlabel=L"k_BT/J", ylabel=L"\langle E \rangle", title="Energy vs Temperature")
    savefig("energy_vs_temperature.png")
    plot(Ts, m_T, label="", xlabel=L"k_BT/J", ylabel=L"\langle m \rangle", title="Magnetization vs Temperature")
    savefig("magnetization_vs_temperature.png")
    plot(Ts, abs_m_T, label="", xlabel=L"k_BT/J", ylabel=L"\langle |m| \rangle", title="Absolute Magnetization vs Temperature")
    savefig("absolute_magnetization_vs_temperature.png")
    plot(Ts, mm_T, label="", xlabel=L"k_BT/J", ylabel=L"\langle \sigma\sigma^\prime \rangle", title="Spin Correlation vs Temperature")
    savefig("spin_correlation_vs_temperature.png")
    plot(Ts, C_T, label="", xlabel=L"k_BT/J", ylabel=L"C", title="Specific Heat vs Temperature")
    savefig("specific_heat_vs_temperature.png")
end

run_ising_MC (generic function with 1 method)

In [113]:
run_ising_MC()

"c:\\Users\\ito23\\Documents\\学院科目\\物性物理学特論Ⅰ\\Julia\\day4\\Mypackage\\specific_heat_vs_temperature.png"

In [ ]:
function compress_expansion(C::NormalizedTensor, P::NormalizedTensor, chi::Int64)
    z, w = inds(C.tensor)

    D, U = eigen(C.tensor, z, w)
    
    n = dim(z)

    p = Index(chi, "p")
    q = Index(chi, "q")

    C_compressed = ITensor(p, q)

    for ii in 1:chi, jj in 1:chi
        C_compressed[p => ii, q => jj] = D[inds(D)[1] => n - ii + 1, inds(D)[2] => n - jj + 1]
    end

In [ ]:
function compress_expansion(C::NormalizedTensor, P::NormalizedTensor, chi::Int64)
    z, w = inds(C.tensor)

    D, U = eigen(C.tensor, z, w)
    
    n = dim(z)

    p = Index(chi, "p")
    q = Index(chi, "q")

    C_compressed = ITensor(p, q)

    for ii in 1:chi, jj in 1:chi
        C_compressed[p => ii, q => jj] = D[inds(D)[1] => n - ii + 1, inds(D)[2] => n - jj + 1]
    end

    NC_compressed = normalize_tensor(C_compressed)
    norm_C = C.norm * NC_compressed.norm

    inds_U = inds(U)
    NU = normalize_tensor(U)

    u = Index(dim(inds(U)[1]), "u")
    v = Index(chi, "v")
    
    U_compressed = ITensor(u, v)

    for ii in 1:n, jj in 1:chi
        U_compressed[u => ii, v => jj] = NU.tensor[inds_U[1] => n - ii + 1, inds_U[2] => n - jj + 1]
    end

    inds_U_compressed = inds(U_compressed)
    inds_P = inds(P.tensor)
    n_u = dim(inds_U_compressed[1])
    m_u = dim(inds_U_compressed[2])

    a = Index(n_u, "a")
    b = Index(m_u, "b")
    c = Index(n_u, "c")
    d = Index(m_u, "d")
    e = Index(dim(inds_P[1]), "e")

    inds_new_U_compressed_l = [a, b]
    inds_new_U_compressed_r = [c, d]
    inds_new_P = [e, a, c]

    U_compressed_l = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_l)
    U_compressed_r = replaceinds(U_compressed, inds_U_compressed, inds_new_U_compressed_r)
    P_b = replaceinds(P.tensor, inds_P, inds_new_P)

    P_compressed = P_b * U_compressed_l * U_compressed_r
    return NormalizedTensor(NC_compressed.tensor, norm_C), NormalizedTensor(NP_compressed.tensor, norm_P)
end